In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
train = pd.read_csv('train_LP.csv')
test = pd.read_csv('test_LP.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train_LP.csv'

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
train['LoanAmount'] = train['LoanAmount'].fillna(train['LoanAmount'].mean())
train['Loan_Amount_Term'] = train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mean())
train['Credit_History'] = train['Credit_History'].fillna(train['Credit_History'].mean())

In [ ]:
train['Gender'] = train['Gender'].fillna(train['Gender'].mode()[0])
train['Married'] = train['Married'].fillna(train['Married'].mode()[0])
train['Dependents'] = train['Dependents'].fillna(train['Dependents'].mode()[0])
train['Self_Employed'] = train['Self_Employed'].fillna(train['Self_Employed'].mode()[0])

In [ ]:
train.head(2)

## Exploratory Data Analysis

In [ ]:
train.columns

In [ ]:
cat_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']
num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
target = ['Loan_Status']

### Univariate Analysis

In [ ]:
for col in cat_cols:
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(111)
    sns.countplot(x = col, data = train, ax = ax)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Frequency Distribution for individual class in {col}')

In [ ]:
sns.countplot(train['Loan_Status'])

In [ ]:
for col in num_cols:
    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax = sns.distplot(train[col], color='m', label=train[col].skew())
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Frequency Distribution for individual class in {col}')
    ax.legend()

### Bivariate Analysis

In [ ]:
sns.pairplot(train[num_cols])

In [ ]:
for cat_col in cat_cols:
    fig = plt.figure(figsize = (30,5))
    j = 1
    for num_col in num_cols:
        ax = fig.add_subplot(1, len(num_cols), j)
        sns.boxplot( x = cat_col, y = num_col, data = train, ax = ax)
        ax.set_xlabel(cat_col)
        ax.set_ylabel(num_col)
        ax.set_title(f'Distribution of {num_col} with respect to {cat_col}')
        j = j+1

In [ ]:
corr = train.corr()
plt.figure(figsize=(14, 7))
sns.heatmap(corr, annot=True, cmap='coolwarm')

### outlier handling

In [ ]:
def handle_outliers(df, var):
    var_data = df[var].values
    q25 = np.percentile(var_data, 25)
    q75 = np.percentile(var_data, 75)
    
    iqr = q75 - q25
    
    lower = q25 - (iqr*1.5)
    upper = q75 + (iqr*1.5)
    
    outliers = [x for x in var_data if x < lower or x > upper]
    
    print(f'Number of Outliers in {var} = {len(outliers)}')
    
    return list(df[(df[var] > upper) | (df[var] < lower)].index)

In [ ]:
train_data = train.copy()

In [ ]:
train.shape

In [ ]:
train_data.shape

In [ ]:
outliers = []
for num_col in num_cols:
    outliers.extend(handle_outliers(train_data, num_col))

outliers = list(set(outliers))

In [ ]:
train_data = train_data.drop(outliers)

In [ ]:
train_data.shape

### Catorigal Encoding

In [ ]:
train_data[cat_cols + num_cols].head()

In [ ]:
pd.get_dummies(train_data[cat_cols + num_cols], columns= cat_cols, drop_first= True)

In [ ]:
train_data_dm = pd.get_dummies(train_data[cat_cols + num_cols + target], columns= cat_cols, drop_first =True)

In [ ]:
train_data_dm.head()

### Data Split

In [ ]:
from sklearn.model_selection import train_test_split
train_data_train, test_data_test = train_test_split(train_data_dm, test_size=0.2, random_state=102)

In [ ]:
train_data_train.shape

In [ ]:
test_data_test.shape

In [ ]:
train_data_dm.shape

In [ ]:
280+70

In [ ]:
x_train = train_data_train.drop(target, axis=1)
y_train = train_data_train[target]

In [ ]:
x_test = test_data_test.drop(target, axis=1)
y_test = test_data_test[target]

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

### Categorical encoding - Target (Label) Variable

In [ ]:
# Lebel Encoding
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(y_train)

In [ ]:
le.classes_

In [ ]:
y_train = le.transform(y_train)

In [ ]:
y_test = le.transform(y_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
len(y_test)

In [ ]:
len(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm